In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
train_images = glob.glob("/content/drive/MyDrive/udemy_異常検知_画像/data/*")
train_data = []

for i in train_images:
  image = cv2.imread(i)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  train_data.append(image)

train_data = np.array(train_data)
train_data = train_data.astype(float)/255

In [ ]:
LEARNING_RATE = 0.001
BATCH_SIZE = 8
EPOCHS = 50

In [ ]:
# エンコーダ
encoder_input = Input(shape=(128,128,3))

x = Conv2D(64, (3,3), activation='relu', padding='same')(encoder_input)
x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(16, (3,3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same')(x)

# デコーダ
x = Conv2D(16, (3,3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2,2))(x)

x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)

x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)

decoded = Conv2D(3,(3,3), activation='sigmoid', padding='same')(x)

autoencoder = Model(encoder_input, decoded)
autoencoder.summary()

In [ ]:
optimizer = Adam(learning_rate = LEARNING_RATE)
autoencoder.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics=["mae", "accuracy"])

autoencoder.fit(train_data, train_data, batch_size=BATCH_SIZE, epochs=EPOCHS)

In [ ]:
test_images = glob.glob("/content/drive/MyDrive/udemy_異常検知_画像/test_data/*")
test_data = []

for i in test_images:
  image = cv2.imread(i)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  test_data.append(image)

test_data = np.array(test_data)
test_data = test_data.astype(float)/255

In [ ]:
reconst_images = autoencoder.predict(test_data)

In [ ]:
diff_images = np.abs(test_data - reconst_images)

In [ ]:
# 差分画像データ
plt.figure(figsize=(10,10))

for i in range(len(diff_images)):
  plt.subplot(3,4,i+1)
  plt.imshow(diff_images[i])
  score = int(np.sum(diff_images[i]))
  plt.title(f"image {i+1}, {score}")

plt.tight_layout()
plt.show()

In [ ]:
np.max(diff_images[0])

In [ ]:
np.max(diff_images[1])

In [ ]:
np.max(diff_images[2])

In [ ]:
np.max(diff_images[3])

In [ ]:
np.max(diff_images[8])

In [ ]:
np.max(diff_images[9])

In [ ]:
# 復元データ
plt.figure(figsize=(10,10))

for i in range(len(reconst_images)):
  plt.subplot(3,4,i+1)
  plt.imshow(reconst_images[i])
  plt.title(f"image {i+1}")

plt.tight_layout()
plt.show()

In [ ]:
diff_images2 = np.where(diff_images < 0.3, 0, 255)

In [ ]:
# 差分画像データ（修正版）
plt.figure(figsize=(10,10))

for i in range(len(diff_images2)):
  plt.subplot(3,4,i+1)
  plt.imshow(diff_images2[i])
  score = int(np.sum(diff_images2[i]))
  plt.title(f"image {i+1}, {score}")

plt.tight_layout()
plt.show()